In [1]:
#pip install -U spacy
#pip install --user en_core_web_sm-3.0.0.tar.gz

In [2]:
#with open("sample.txt", "r") as fh:
#with open("hdfs-audit.log", "r") as fh:
with open("sample1.txt", "r") as fh:
    fstring = fh.readlines()
print(len(fstring))

61


In [3]:
import re

file_lst=[]
with open("sample1.txt","r") as fh:
    for line in fh:
        res = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
        ent_tup=()
        ent_lst=[]
        file_tup=[]
        ent_dict={"entities":[]}
        for m in res.finditer(line):
            ent_tup = (m.start(),m.end(),"IP")
            ent_lst.append(ent_tup)
        ent_dict["entities"] = ent_lst
        file_tup = (line,ent_dict)
        file_lst.append(file_tup)        
        #print(file_tup)
print(len(file_lst))
#print(file_lst)

61


In [4]:
import spacy
import en_core_web_sm as en
print(spacy.__version__)
print(en.__version__)

3.0.5
3.0.0


In [5]:
#from spacy.language import Language
#@Language.component("info_component")
#def my_component(doc):
#    print(f"After tokenization, this doc has {len(doc)} tokens.")
#    print("The part-of-speech tags are:", [token.pos_ for token in doc])
#    if len(doc) < 10:
#        print("This is a pretty short document.")
#    return doc

In [6]:
#source_nlp = spacy.load("en_core_web_sm")
#print(source_nlp.pipe_names)
##mer = source_nlp.get_pipe("ner")
##print(mer)
#nlp = spacy.blank("en")
#nlp.add_pipe("ner",source=source_nlp)
#print(nlp.pipe_names)

In [7]:
#Add new pipe - "print_info" into nlp object
#nlp.add_pipe("info_component", name="print_info", first=True)
#print(nlp.pipe_names)  # ['tagger', 'parser', 'ner', 'print_info']
#doc = nlp("This is a sentence.")

In [8]:
nlp = spacy.load("en_core_web_sm")
print(nlp)
doc = nlp("Australia")
for ent in doc.ents:
    print(ent.text,ent.start_char,ent.end_char,ent.label_)

Australia 0 9 GPE


In [9]:
print(nlp.pipe_names)
ner = nlp.get_pipe("ner")
print(ner)

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


In [10]:
for _, annotations in file_lst:
    for ent in annotations.get("entities"):
        #print(ent[2])
        ner.add_label(ent[2])

In [11]:
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(disable_pipes)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']


In [12]:
import warnings
warnings.filterwarnings("ignore")
import random
from spacy.util import minibatch, compounding
from spacy.training import Example

with nlp.disable_pipes(*disable_pipes):
    
    optimizer = nlp.resume_training()
    
    for iteration in range(100):
        
        random.shuffle(file_lst)
        losses={}
        
        for batch in minibatch(file_lst, size=5):
            for text,annotation in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc,annotation)
                #print(example)
                nlp.update([example],losses=losses,drop=0.3,sgd=optimizer)
        print(f'Loss in iter {iteration} is {losses}')

Loss in iter 0 is {'ner': 40.1455352544494}
Loss in iter 1 is {'ner': 12.038721134380337}
Loss in iter 2 is {'ner': 0.04384283711739288}
Loss in iter 3 is {'ner': 0.16801913500963014}
Loss in iter 4 is {'ner': 0.0003263983282073047}
Loss in iter 5 is {'ner': 0.0003230234689985609}
Loss in iter 6 is {'ner': 0.0005080485768495294}
Loss in iter 7 is {'ner': 1.7935567961747136e-05}
Loss in iter 8 is {'ner': 3.517261583522815e-06}
Loss in iter 9 is {'ner': 0.008232498792038332}
Loss in iter 10 is {'ner': 0.00042963463771303616}
Loss in iter 11 is {'ner': 0.0007365443208581817}
Loss in iter 12 is {'ner': 1.0504157268138657e-06}
Loss in iter 13 is {'ner': 3.331751463274324e-09}
Loss in iter 14 is {'ner': 1.9195951561787953e-07}
Loss in iter 15 is {'ner': 6.342007756991866e-08}
Loss in iter 16 is {'ner': 6.2457642054939905e-06}
Loss in iter 17 is {'ner': 9.028794868388489e-08}
Loss in iter 18 is {'ner': 2.824638424029554e-05}
Loss in iter 19 is {'ner': 1.1148561750112597e-08}
Loss in iter 20 i

In [13]:
for text, _ in file_lst:
    #print(text)
    doc=nlp(text)
    print("Entities",[(ent.text,ent.label_) for ent in doc.ents])

Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('10.45.149.37', 'IP'), ('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities [('13.24.199.197', 'IP')]
Entities []
Entities []
Entities []
Entities []
Ent

In [15]:
from spacy import displacy
doc = nlp("the IP which is causing issue in 192.167.23.23")
for ent in doc.ents:
    print(ent)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(nlp(doc.text),style='ent',jupyter=True)

192.167.23.23
192.167.23.23 33 46 IP
